In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
import numpy as np
import pathlib

AUTOTUNE = tf.data.experimental.AUTOTUNE

tf.__version__

'2.0.0'

In [3]:
print("tensorflow version check : ", tf.__version__)
print("gpu check : ", tf.test.is_gpu_available())

tensorflow version check :  2.0.0
gpu check :  True


In [4]:
# import zipfile
# file = 'image_train.zip'

# zipRef = zipfile.ZipFile(file, 'r')
# zipRef.extractall()
# zipRef.close()

In [5]:
train_dir = pathlib.Path('Training')
private_test_dir = pathlib.Path('PrivateTest')
public_test_dir = pathlib.Path('PublicTest')

In [6]:
list_ds = tf.data.Dataset.list_files(str(train_dir/'*/*'))
val_list_ds = tf.data.Dataset.list_files(str(public_test_dir/'*/*'))

In [7]:
#확인하기
for f in list_ds.take(5) : 
    print(f)

tf.Tensor(b'Training/Sad/9382.jpg', shape=(), dtype=string)
tf.Tensor(b'Training/Surprise/22703.jpg', shape=(), dtype=string)
tf.Tensor(b'Training/Neutral/1578.jpg', shape=(), dtype=string)
tf.Tensor(b'Training/Happy/13763.jpg', shape=(), dtype=string)
tf.Tensor(b'Training/Happy/780.jpg', shape=(), dtype=string)


In [8]:
CLASS_NAMES = np.array([item.name for item in train_dir.glob('*')])
CLASS_NAMES

array(['Surprise', 'Fear', 'Disgust', 'Happy', 'Neutral', 'Sad', 'Angry'],
      dtype='<U8')

In [9]:
for i in CLASS_NAMES:
    print(i)
    print(len(list(train_dir.glob('{}/*.jpg'.format(i)))))

Surprise
3171
Fear
4097
Disgust
436
Happy
7215
Neutral
4965
Sad
4830
Angry
3995


In [10]:
CLASS_NAME_TO_ID = {}
for id, name in enumerate(CLASS_NAMES):
    CLASS_NAME_TO_ID[name] = id
    
print(CLASS_NAME_TO_ID)

{'Surprise': 0, 'Fear': 1, 'Disgust': 2, 'Happy': 3, 'Neutral': 4, 'Sad': 5, 'Angry': 6}


In [11]:
keys = list(CLASS_NAME_TO_ID.keys())
ids = list(CLASS_NAME_TO_ID.values())
print(keys,ids)

['Surprise', 'Fear', 'Disgust', 'Happy', 'Neutral', 'Sad', 'Angry'] [0, 1, 2, 3, 4, 5, 6]


In [12]:
table = tf.lookup.StaticHashTable(
    initializer= tf.lookup.KeyValueTensorInitializer(
    keys= tf.constant(keys),
    values=tf.constant(ids),),
    default_value=tf.constant(-1),
    name="class_weight")

In [13]:
def get_label(file_path) :
    parts = tf.strings.split(file_path, '/')
    return table.lookup(parts[-2])

def decode_img(img):
    img = tf.image.decode_jpeg(img,channels=1)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return tf.image.resize(img,[IMG_WIDTH,IMG_HEIGHT])

def process_path(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [14]:
BATCH_SIZE = 128
IMG_HEIGHT = 96
IMG_WIDTH = 96

labeled_ds = list_ds.shuffle(10000).map(process_path, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE)

In [15]:
for image, label in labeled_ds.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

Image shape:  (128, 96, 96, 1)
Label:  [4 0 3 1 3 3 3 3 6 3 6 3 4 4 4 5 4 5 5 1 3 3 4 5 4 3 3 1 3 3 0 5 1 5 3 4 1
 6 3 3 3 0 0 3 1 1 4 3 0 5 6 6 1 1 4 0 4 3 6 3 6 0 0 5 6 4 6 4 3 3 0 4 1 1
 4 3 3 1 6 3 5 4 5 4 5 3 6 0 6 1 3 5 1 3 1 0 4 0 3 4 4 6 3 6 0 1 0 4 5 6 5
 6 5 3 4 1 4 1 4 4 5 5 0 5 1 0 0 4]


In [16]:
labeled_ds.take(2)

<DatasetV1Adapter shapes: ((None, 96, 96, 1), (None,)), types: (tf.float32, tf.int32)>

In [17]:
inputs = keras.Input(shape=(96, 96,1), name='img')

feature = tf.keras.layers.Conv2D(8, 3, padding = 'same', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3))(inputs)
feature = tf.keras.layers.BatchNormalization()(feature)
feature = tf.keras.layers.ReLU()(feature)
feature = tf.keras.layers.Conv2D(8, 3, padding='same')(feature)
feature = tf.keras.layers.BatchNormalization()(feature)
feature = tf.keras.layers.ReLU()(feature)
feature = tf.keras.layers.Conv2D(8, 3, padding='same')(feature)
feature = tf.keras.layers.BatchNormalization()(feature)
feature = tf.keras.layers.ReLU()(feature)
feature = tf.keras.layers.MaxPooling2D()(feature)
feature = tf.keras.layers.Dropout(0.25)(feature)


feature = tf.keras.layers.Conv2D(32, 3, padding='same')(feature)
feature = tf.keras.layers.BatchNormalization()(feature)
feature = tf.keras.layers.ReLU()(feature)
feature = tf.keras.layers.Conv2D(32, 3, padding='same')(feature)
feature = tf.keras.layers.BatchNormalization()(feature)
feature = tf.keras.layers.ReLU()(feature)
feature = tf.keras.layers.Conv2D(32, 3, padding='same')(feature)
feature = tf.keras.layers.BatchNormalization()(feature)
feature = tf.keras.layers.ReLU()(feature)
feature = tf.keras.layers.MaxPooling2D()(feature)
feature = tf.keras.layers.Dropout(0.25)(feature)


feature = tf.keras.layers.Conv2D(64, 3, padding='same')(feature)
feature = tf.keras.layers.BatchNormalization()(feature)
feature = tf.keras.layers.ReLU()(feature)
feature = tf.keras.layers.Conv2D(64, 3, padding='same')(feature)
feature = tf.keras.layers.BatchNormalization()(feature)
feature = tf.keras.layers.ReLU()(feature)
feature = tf.keras.layers.Conv2D(64, 3, padding='same')(feature)
feature = tf.keras.layers.BatchNormalization()(feature)
feature = tf.keras.layers.ReLU()(feature)
feature = tf.keras.layers.MaxPooling2D()(feature)
feature = tf.keras.layers.Dropout(0.25)(feature)

feature = tf.keras.layers.Conv2D(128, 3, padding='same')(feature)
feature = tf.keras.layers.BatchNormalization()(feature)
feature = tf.keras.layers.ReLU()(feature)
feature = tf.keras.layers.Conv2D(128, 3, padding='same')(feature)
feature = tf.keras.layers.BatchNormalization()(feature)
feature = tf.keras.layers.ReLU()(feature)
feature = tf.keras.layers.Conv2D(128, 3, padding='same')(feature)
feature = tf.keras.layers.BatchNormalization()(feature)
feature = tf.keras.layers.ReLU()(feature)
feature = tf.keras.layers.MaxPooling2D()(feature)
feature = tf.keras.layers.Dropout(0.25)(feature)

#feature = tf.keras.layers.Flatten()(feature)
feature = tf.keras.layers.GlobalAveragePooling2D()(feature)
feature = tf.keras.layers.Dense(256, activation='relu')(feature)
feature = tf.keras.layers.Dense(128, activation='relu')(feature)
outputs = tf.keras.layers.Dense(7, activation='softmax')(feature)

model = keras.Model(inputs=inputs, outputs=outputs)

In [18]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img (InputLayer)             [(None, 96, 96, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 96, 96, 8)         80        
_________________________________________________________________
batch_normalization (BatchNo (None, 96, 96, 8)         32        
_________________________________________________________________
re_lu (ReLU)                 (None, 96, 96, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 96, 8)         584       
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 96, 8)         32        
_________________________________________________________________
re_lu_1 (ReLU)               (None, 96, 96, 8)         0     

In [19]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

In [20]:
print(len(model.trainable_variables))

print('========================Training===============================')

model.fit(labeled_ds, epochs=20)

54
========================Training===============================
Epoch 1/20
      1/Unknown - 4s 4s/step

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model/conv2d/Conv2D (defined at /home/ubuntu/anaconda3/envs/juns/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_distributed_function_5487]

Function call stack:
distributed_function
